In [2]:
!pip install tqdm
!pip install langchain
!pip install langchain-community
!pip install pypdf
!pip install gpt4all
# !pip install sentence_transformers
# !pip install faiss-gpu
!pip install langchain-qdrant

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.4/396.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/2

In [3]:
import os
from glob import glob

from google.colab import userdata

from tqdm import tqdm
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings.gpt4all import GPT4AllEmbeddings
# from langchain.vectorstores import FAISS
from langchain_qdrant import QdrantVectorStore

In [ ]:
from google.colab import userdata
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=900,
    chunk_overlap=20,
    length_function=len
)

In [ ]:
def get_pages_from_pdf_document(pdffilepath):
    loader = PyPDFLoader(pdffilepath)
    pages = loader.load_and_split(text_splitter=text_splitter)
    return pages


def get_pages_from_pdf_documents(directory):
    pages = []
    for pdffilepath in tqdm(glob(os.path.join(directory, '*.pdf'))):
        this_pages = get_pages_from_pdf_document(pdffilepath)
        if this_pages is not None and len(this_pages) > 0:
            pages += this_pages
    return pages


def iterate_list_pdfnames(directory):
    for pdffilepath in tqdm(glob(os.path.join(directory, '*.pdf'))):
        basename = os.path.basename(pdffilepath)
        yield basename

In [ ]:
def main(corpusdir, embed_model_name, qdrant_url, qdrant_api_key):
    embedding_model = GPT4AllEmbeddings(model_name=embed_model_name)
    # embedding_model = HuggingFaceEmbeddings(model_name=embed_model_name)
    # if embedding_model.client.tokenizer.pad_token is None:
        # embedding_model.client.tokenizer.pad_token = embedding_model.client.tokenizer.eos_token

    pages = get_pages_from_pdf_documents(corpusdir)
    # db = FAISS.from_documents(pages, embedding_model)
    qdrant = QdrantVectorStore.from_documents(
        pages,
        embedding_model,
        url=qdrant_url,
        prefer_grpc=True,
        api_key=qdrant_api_key,
        collection_name='apologetics'
    )

In [ ]:
corpusdir = '/content/drive/MyDrive/Apologetics Corpus'
qdrant_url = userdata.get('APOLOGETICS_QDRANT_URL')
qdrant_api_key = userdata.get('LANGSMITH_API_KEY')
embed_model_name = 'all-MiniLM-L6-v2.gguf2.f16.gguf'
# embed_model_name = 'sentence-transformers/all-MiniLM-L6-v2'
vectorstoredir = '/content/drive/MyDrive/Apologetics Vectorstore'

In [ ]:
f = open(os.path.join(vectorstoredir, 'booklist_qdrant.txt'), 'w')
for line in iterate_list_pdfnames(corpusdir):
  f.write(line + '\n')
f.close()

100%|██████████| 37/37 [00:00<00:00, 178378.45it/s]


In [ ]:
main(corpusdir, embed_model_name, qdrant_url, qdrant_api_key)

100%|██████████| 37/37 [26:35<00:00, 43.13s/it]
